## Intuición: buscar el mínimo del costo

Antes de entrar a la matemática del **Backpropagation**, pensemos en un caso muy simple: "Una red con un solo parámetro $W$".

Nuestro objetivo es encontrar el valor de $W$ que minimiza la función de costo $C(W)$.

Podemos imaginar a $C(W)$ como una colina con valles y cumbres.  
Lo que buscamos es el punto más bajo posible, el mínimo absoluto o, al menos, un buen mínimo local donde el costo (error) sea pequeño.

A primera vista, parece sencillo. Si conocemos la expresión exacta de $C(W)$, basta con derivarla e igualar la derivada a cero para hallar los puntos críticos.  
Luego, aplicando la segunda derivada, podemos confirmar si el punto encontrado corresponde a un mínimo (concavidad hacia arriba) o a un máximo (concavidad hacia abajo).

Por ejemplo, si la función de costo estuviera representada por:

$$
C(W) = -0.07968W^5 + 0.48721W^4 + 0.28001W^3 - 4.65127W^2 + 15.7
$$

entonces su primera derivada sería:

$$
C'(W) = -0.3984W^4 + 1.94884W^3 + 0.84003W^2 - 9.30254W
$$

y su segunda derivada:

$$
C''(W) = -1.5936W^3 + 5.84652W^2 + 1.68006W - 9.30254
$$

Con esas tres expresiones podríamos hallar analíticamente los puntos donde la función tiene máximos o mínimos y determinar la curvatura local para confirmar que se trata efectivamente de un mínimo. Tal cual se verá en la siguiente simulación:

In [12]:
### SIMULACIÓN ###

# El siguiente código es únicamente con fines de simular el ejemplo

# Denle al botón "play" (►) de abajo para ver la animación

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

a5 = -0.07968
a4 =  0.48721
a3 =  0.28001
a2 = -4.65127
a1 =  0.0
a0 =  15.7

def f(x):
    return a5*x**5 + a4*x**4 + a3*x**3 + a2*x**2 + a1*x + a0

def fp(x):
    return 5*a5*x**4 + 4*a4*x**3 + 3*a3*x**2 + 2*a2*x + a1

def fpp(x):
    return 20*a5*x**3 + 12*a4*x**2 + 6*a3*x + 2*a2

xmin, xmax = -3.5, 4.0
X = np.linspace(xmin, xmax, 800)
Y = f(X)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(X, Y, linewidth=3, label='C(W)')
ax.axvline(3, linestyle='--', linewidth=1)
ax.axhline(1.5, linestyle=':', linewidth=1)
ax.scatter([3], [1.5], s=80)
ax.set_xlim(xmin, xmax)
ax.set_ylim(-5, 18)
ax.set_xlabel('W')
ax.set_ylabel('C(W)')
ax.grid(True)

line_tan, = ax.plot([], [], color='black', linewidth=2.5, label='Tangente')
pt, = ax.plot([], [], 'ro', markersize=8)
info = ax.text(-3.3, 16.5, '', fontsize=11, bbox=dict(facecolor='white', alpha=0.75))

def init():
    line_tan.set_data([], [])
    pt.set_data([], [])
    info.set_text('')
    return line_tan, pt, info

def update(frame):
    x0 = -3.5 + frame * 0.05
    if x0 > 3.0:
        x0 = 3.0
    y0 = f(x0)
    m = fp(x0)
    c2 = fpp(x0)
    Yt = y0 + m*(X - x0)
    line_tan.set_data(X, Yt)
    pt.set_data([x0], [y0])

    conc = "∪ mínimo" if c2 > 0 else "∩ máximo" if c2 < 0 else "plano"
    tol = 1e-3
    if abs(m) < tol:
        trend = "estacionario"
    elif m > 0:
        trend = "aumenta"
    else:
        trend = "disminuye"

    ax.set_title(f'Tangente en W={x0:.2f}, C(W)={y0:.2f}')
    info.set_text(f"C'(W)={m:.3f}\nC''(W)={c2:.3f}\nConcavidad: {conc}\nError: {trend}")
    return line_tan, pt, info

frames = int((xmax - xmin) / 0.05)
ani = animation.FuncAnimation(fig, update, init_func=init, frames=frames, interval=30, blit=True)

html = HTML(ani.to_jshtml())
display(html)
plt.close()

Output hidden; open in https://colab.research.google.com to view.

## ¿Y por qué esto no funciona con redes neuronales?

El problema es que, en redes neuronales, el costo no depende de un solo parámetro, sino de **miles o millones de pesos y sesgos**:

$$
C = f(W_1, W_2, W_3, \dots, W_n)
$$

A eso se suma que:

1. La relación entre esos parámetros y el costo es **altamente no lineal**.  
2. La función resultante tiene una superficie llena de **mínimos locales, mesetas y regiones planas**.  
3. Y lo más importante: **no conocemos de forma explícita la expresión matemática exacta de $C$**.  
   Solo sabemos cómo calcular su valor numérico en la salida de la red tras un *forward pass*, pero no tenemos una ecuación cerrada que podamos derivar directamente.  
   En otras palabras, estamos "a ciegas" respecto a la forma completa de la función.

Por eso, aunque la idea de usar derivadas no está equivocada, no podemos aplicarla de manera simbólica y directa como en el ejemplo anterior.  
En su lugar, necesitaremos un método que calcule las derivadas parciales de forma numérica y eficiente, incluso cuando la función es demasiado compleja para expresarse explícitamente.  
Ahí es donde entra el **Backpropagation**.

# Regla de la cadena en el backpropagation


## 1. Propagación hacia adelante

Durante la *forward pass*, los valores fluyen desde la entrada hacia la salida:

$$
z^{(L)} = W^{(L)} a^{(L-1)} + b^{(L)}
$$

$$
a^{(L)} = f(z^{(L)})
$$

donde:  
- $a^{(L-1)}$ son las activaciones de la capa anterior  
- $W^{(L)}$ y $b^{(L)}$ son los pesos y sesgos de la capa actual  
- $f$ es la función de activación (por ejemplo *sigmoid*, *ReLU*, *Tanh*, etc.)

---

## 2. Regla de la cadena

El objetivo del *backpropagation* es calcular cómo cambia el error (costo) con respecto a cada parámetro de la red (peso o bias).

Si una variable depende de otra, y esa depende de otra más, la regla de la cadena indica:

$$
\frac{dC}{dx} = \frac{dC}{dy} \cdot \frac{dy}{dx}
$$

Donde $y$ es una función que depende de $x$.

<br>

Y en cadenas más largas:

$$
\frac{dC}{dx} = \frac{dC}{dz} \cdot \frac{dz}{dy} \cdot \frac{dy}{dx}
$$

Donde $z$ es una función que depende de $y$, y $y$ es una función que depende de $x$.

En redes neuronales, esto significa que el gradiente del error se propaga multiplicando derivadas capa a capa hacia atrás.


In [13]:
### SIMULACIÓN ###

# El siguiente código es únicamente con fines de simular el ejemplo

# Denle al botón "play" (►) de abajo para ver la animación

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

matplotlib.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams["animation.embed_limit"] = 25

def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def chain_cost(x, t):
    y = sigmoid(x)
    z = y**2
    C = 0.5*(z - t)**2
    dC_dz = (z - t)
    dz_dy = 2.0*y
    dy_dx = y*(1.0-y)
    dC_dx = dC_dz*dz_dy*dy_dx
    return y, z, C, dC_dz, dz_dy, dy_dx, dC_dx

def set_vline(vline, ax, x):
    vline.set_data([x, x], ax.get_ylim())

t_target = 0.6
xgrid = np.linspace(-6, 6, 300)
ygrid = sigmoid(xgrid)
zgrid = ygrid**2
Cgrid = 0.5*(zgrid - t_target)**2

fig, ax = plt.subplots(figsize=(10, 6))
line_y, = ax.plot(xgrid, ygrid, lw=2, label="y=σ(x)")
line_z, = ax.plot(xgrid, zgrid, lw=2, ls="--", label="z=y^2")
line_C, = ax.plot(xgrid, Cgrid, lw=2, ls=":", label="C(x)")
vline, = ax.plot([], [], lw=1)
dot_y, = ax.plot([], [], "o", ms=6)
dot_z, = ax.plot([], [], "o", ms=6)
dot_C, = ax.plot([], [], "o", ms=6)
txt = ax.text(0.02, 0.98, "", transform=ax.transAxes, va="top")
ax.set_xlim(-6, 6)
ax.set_ylim(-0.1, max(Cgrid.max(), zgrid.max(), ygrid.max())+0.2)
ax.set_title("Regla de la cadena escalar")
ax.set_xlabel("x")
ax.set_ylabel("Valores")
ax.legend(loc="lower right")

frames = 140
xs = np.linspace(-6, 6, frames)

def init():
    set_vline(vline, ax, xs[0])
    dot_y.set_data([], [])
    dot_z.set_data([], [])
    dot_C.set_data([], [])
    txt.set_text("")
    return line_y, line_z, line_C, vline, dot_y, dot_z, dot_C, txt

def update(k):
    x = xs[k]
    y, z, C, dC_dz, dz_dy, dy_dx, dC_dx = chain_cost(x, t_target)
    set_vline(vline, ax, x)
    dot_y.set_data([x], [y])
    dot_z.set_data([x], [z])
    dot_C.set_data([x], [C])
    txt.set_text(
        f"x={x:.3f}\ny={y:.3f}\nz={z:.3f}\nC={C:.3f}\n"
        f"∂C/∂z={dC_dz:.3f}\n∂z/∂y={dz_dy:.3f}\n∂y/∂x={dy_dx:.3f}\n"
        f"∂C/∂x={dC_dx:.3f}"
    )
    return line_y, line_z, line_C, vline, dot_y, dot_z, dot_C, txt

anim = animation.FuncAnimation(fig, update, init_func=init, frames=frames, interval=40, blit=False)
html1 = HTML(anim.to_jshtml())
display(html1)
plt.close()

Output hidden; open in https://colab.research.google.com to view.

## 3. Propagación del error

Sea $C$ la función de costo de la red. Buscamos hallar cómo varía este error con respecto a un parámetro de la red que puede ser un peso ($w$) o un bias ($b$).

Entonces nuestro objetivo es hallar:
$$
\frac{\partial C}{\partial w^{(L)}_{ij}}
$$

Y también:

$$
\frac{\partial C}{\partial b^{(L)}_{i}}
$$

Donde $w^{(L)}_{ij}$ es el peso que conecta la neurona número $j$ de la capa $L-1$ con la neurona número $i$ de la capa $L$.

Y $b^{(L)}_{i}$ es el bias de la neurona numero $i$ de la capa $L$.

<br>

El problema es que no existe manera de hacer esas derivadas de manera directa, ya que existe un cadena de dependencias intermedias que no se puede ignorar.

La función $C$ no depende directamente de $w$ o $b$, sino que depende de la salida de la función de activación $a^{(L)}$:

$$
C = f(a^{(L)})
$$

Esta a su vez, depende de la suma ponderada $z^{(L)}_{i}$:

$$
a^{(L)} = f(z^{(L)})
$$

Y esta última, recién es una función que depende de $w$ y $b$:

$$
z^{(L)} = f(w, b)
$$

Por lo que me queda lo siguiente:

$$
C(a^{(L)}(z^{(L)}(w, b)))
$$

<br>

Debido a esto, la solución más óptima para calcular $\frac{\partial C}{\partial w}$ y $\frac{\partial C}{\partial b}$ es usando la regla de la cadena:

$$
\frac{\partial C}{\partial w^{(L)}} = \frac{\partial C}{\partial a^{(L)}} \cdot \frac{\partial a^{(L)}}{\partial z^{(L)}} \cdot \frac{\partial z^{(L)}}{\partial w^{(L)}}
$$

$$
\frac{\partial C}{\partial b^{(L)}} = \frac{\partial C}{\partial a^{(L)}} \cdot \frac{\partial a^{(L)}}{\partial z^{(L)}} \cdot \frac{\partial z^{(L)}}{\partial b^{(L)}}
$$

---

## 4. Vamos con las derivadas parciales

Función de coste:

$$
C(a^L_j) = \frac{1}{2} \sum_j (y_j - a^L_j)^2
$$

Y su derivada con respecto a la activación de salida:

$$
\frac{\partial C}{\partial a^L_j} = (a^L_j - y_j)
$$

<br>

Función de activación: Sigmoide

$$
a^L(z^L) = \frac{1}{1 + e^{-z^L}}
$$

Y su derivada con respecto a la entrada $z^L$:

$$
\frac{\partial a^L}{\partial z^L} = a^L(z^L) \cdot (1 - a^L(z^L))
$$

<br>

Derivando la suma ponderada:

$$
z^L(w, b) = \sum_i a_i^{L-1} w_i^L + b^L
$$

Derivadas parciales:

$$
\frac{\partial z^{(L)}}{\partial w_i^{(L)}} = a_i^{L-1}
$$

$$
\frac{\partial z^{(L)}}{\partial b^{(L)}} = 1
$$

<br>

Finalmente:
$$
\frac{\partial C}{\partial W^{(L)}} = \frac{\partial C}{\partial a^{(L)}} \cdot \frac{\partial a^{(L)}}{\partial z^{(L)}} \cdot \frac{\partial z^{(L)}}{\partial w^{(L)}} = (a^{(L)}_j - y_j) \cdot a^{(L)}(z^{(L)}) \cdot (1 - a^{(L)}(z^{(L)})) \cdot a_i^{(L-1)}
$$

$$
\frac{\partial C}{\partial b^{(L)}} = \frac{\partial C}{\partial a^{(L)}} \cdot \frac{\partial a^{(L)}}{\partial z^{(L)}} \cdot \frac{\partial z^{(L)}}{\partial b^{(L)}} = (a^{(L)}_j - y_j) \cdot a^{(L)}(z^{(L)}) \cdot (1 - a^{(L)}(z^{(L)})) \cdot 1
$$

<br>

NOTA: Recordar que $a^{(L)}(z^{(L)})$ es una función y $a^{(L)}_j$ es un valor escalar de la salida de esa función.


In [14]:
### SIMULACIÓN ###

# El siguiente código es únicamente con fines de simular el ejemplo

# Denle al botón "play" (►) de abajo para ver la animación

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

matplotlib.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams["animation.embed_limit"] = 25

rng = np.random.default_rng(7)
m = 60
X = rng.normal(size=(m, 2))
true_w = np.array([[2.0, -1.0]])
true_b = np.array([0.5])
Y = 1.0/(1.0+np.exp(-(X @ true_w.T + true_b))) > 0.5
Y = Y.astype(float)

W = rng.normal(scale=0.5, size=(1, 2))
b = np.array([0.0])
eta = 0.5

def forward(X, W, b):
    Z = X @ W.T + b
    A = 1.0/(1.0+np.exp(-Z))
    return Z, A

def loss(A, Y):
    return 0.5*np.mean((A - Y)**2)

def grads(X, A, Y):
    m = X.shape[0]
    dC_dA = (A - Y)
    dA_dZ = A*(1.0-A)
    dC_dZ = dC_dA*dA_dZ
    dC_dW = (dC_dZ.T @ X)/m
    dC_db = dC_dZ.mean(axis=0)
    return dC_dW, dC_db, dC_dZ

steps = 100

fig, (ax_left, ax_right) = plt.subplots(1, 2, figsize=(13, 6), gridspec_kw={"width_ratios": [3, 3]})

pos = Y.ravel() > 0.5
ax_left.scatter(X[pos,0], X[pos,1], s=20, label="Clase 1")
ax_left.scatter(X[~pos,0], X[~pos,1], s=20, label="Clase 0")
line_boundary, = ax_left.plot([], [], lw=2, label="Frontera")
txt_left = ax_left.text(0.02, 0.98, "", transform=ax_left.transAxes, va="top")
ax_left.set_title("Ultima capa: frontera y coste")
ax_left.set_xlabel("x1")
ax_left.set_ylabel("x2")
ax_left.set_xlim(X[:,0].min()-1, X[:,0].max()+1)
ax_left.set_ylim(X[:,1].min()-1, X[:,1].max()+1)
ax_left.legend(loc="lower right")

ax_right.set_title("Costo vs época")
ax_right.set_xlabel("Época")
ax_right.set_ylabel("Costo")
ax_right.set_xlim(0, steps)
ax_right.set_ylim(0.04, 0.12)
line_cost, = ax_right.plot([], [], lw=2)
dot_cost, = ax_right.plot([], [], "o", ms=6)
txt_right = ax_right.text(0.02, 0.98, "", transform=ax_right.transAxes, va="top")

cost_hist = []

def init():
    line_boundary.set_data([], [])
    txt_left.set_text("")
    line_cost.set_data([], [])
    dot_cost.set_data([], [])
    txt_right.set_text("")
    return line_boundary, txt_left, line_cost, dot_cost, txt_right

def update(t):
    global W, b
    Z, A = forward(X, W, b)
    C = loss(A, Y)
    dW, db, _ = grads(X, A, Y)
    W = W - eta*dW
    b = b - eta*db
    xx = np.linspace(ax_left.get_xlim()[0], ax_left.get_xlim()[1], 200)
    if abs(W[0,1]) < 1e-6:
        yy = np.full_like(xx, -b[0])
    else:
        yy = -(W[0,0]*xx + b[0])/W[0,1]
    line_boundary.set_data(xx, yy)
    txt_left.set_text(f"W={W.ravel()}\nb={b}\nC={C:.5f}")

    cost_hist.append(float(C))
    xs = np.arange(len(cost_hist))
    ys = np.array(cost_hist)
    line_cost.set_data(xs, ys)
    if len(xs) > 0:
        dot_cost.set_data([xs[-1]], [ys[-1]])
    txt_right.set_text(f"Época={t+1}\nC={C:.5f}")

    return line_boundary, txt_left, line_cost, dot_cost, txt_right

anim = animation.FuncAnimation(fig, update, init_func=init, frames=steps, interval=40, blit=False)
html2 = HTML(anim.to_jshtml())
display(html2)
plt.close()

Output hidden; open in https://colab.research.google.com to view.

## 5. Error imputado a una neurona

Hemos visto cómo podemos saber cómo influye el valor de un parámetro en el error obtenido. ¿Pero cómo podemos saber por cuál de los caminos o ramificaciones de la red encontrar ese parámetro de manera más rápida?

Para esto debemos encontrar la responsabilidad de cada neurona en el error hallando:
$$
\delta^{(L)} = \frac{\partial C}{\partial z^{(L)}} = \frac{\partial C}{\partial a^{(L)}} \cdot \frac{\partial a^{(L)}}{\partial z^{(L)}} = (a^{(L)}_j - y_j) \cdot a^{(L)}(z^{(L)}) \cdot (1 - a^{(L)}(z^{(L)}))
$$

Por lo que ahora para hallar la derivada del costo con respecto a los parametros sería:

$$
\frac{\partial C}{\partial W^{(L)}} = \delta^{(L)} \cdot a_i^{(L-1)}
$$

$$
\frac{\partial C}{\partial b^{(L)}} = \delta^{(L)}
$$


---

## 6. Propagación del error hacia capas anteriores

En una capa oculta $L-1$, la neurona no "ve" el error directamente.  
Su salida solo afecta al error de la capa siguiente $L$.  
Por tanto, la variación del costo respecto a su entrada $z^{(L-1)}$ se obtiene a través de los pesos y errores de la capa siguiente:

$$
\delta^{(L-1)} = \frac{\partial C}{\partial z^{(L-1)}}
$$

Aplicando la regla de la cadena:

$$
\delta^{(L-1)} = \frac{\partial C}{\partial z^{(L-1)}} = \frac{\partial C}{\partial z^{(L)}_j} \cdot \frac{\partial z^{(L)}_j}{\partial a^{(L-1)}} \cdot \frac{\partial a^{(L-1)}}{\partial z^{(L-1)}}
$$

Sabiendo que:

$$
\frac{\partial z^{(L)}_j}{\partial a^{(L-1)}_i} = w^{(L)}_{ji} = W^{(L)}
$$

Y que:

$$
\frac{\partial C}{\partial z^{(L)}_j} = \delta^{(L)}_j
$$

Entonces:

$$
\delta^{(L-1)}_i = \delta^{(L)}_j \cdot W^{(L)} \cdot \frac{\partial a^{(L-1)}}{\partial z^{(L-1)}}
$$

---

## 7. Gradientes de pesos y bias en capas ocultas

Una vez obtenido el error local de cada neurona $\delta^{(L-1)}$, los gradientes se calculan de la misma forma que en la última capa:

$$
\frac{\partial C}{\partial W^{(L-1)}} = \delta^{(L-1)} \cdot a^{(L-2)}
$$

$$
\frac{\partial C}{\partial b^{(L-1)}} = \delta^{(L-1)}
$$

Este proceso se repite hacia atrás capa por capa hasta llegar a la primera.

In [15]:
### SIMULACIÓN ###

# El siguiente código es únicamente con fines de simular el ejemplo

# Denle al botón "play" (►) de abajo para ver la animación

# Simulación para una red 2-2-1. Es decir, 2 neuronas en la capa de entrada, una capa oculta de 2 neuronas y 1 neurona en la capa de salida.

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

matplotlib.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams["animation.embed_limit"] = 25

rng = np.random.default_rng(3)
X = rng.normal(size=(80, 2))

w1 = np.array([[1.2, -0.7],
               [-0.9,  1.0]], dtype=float)
b1 = np.array([0.0, 0.0], dtype=float)
w2 = np.array([[0.8, -0.6]], dtype=float)
b2 = np.array([0.0], dtype=float)

Y = (X[:,0]**2 + X[:,1]**2 > 1.3).astype(float).reshape(-1,1)

def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def forward_2_2_1(X, w1, b1, w2, b2):
    z1 = X @ w1.T + b1
    a1 = sigmoid(z1)
    z2 = a1 @ w2.T + b2
    a2 = sigmoid(z2)
    return z1, a1, z2, a2

def loss_mse(a2, Y):
    return 0.5*np.mean((a2 - Y)**2)

def backprop_2_2_1(X, Y, z1, a1, z2, a2, w2):
    m = X.shape[0]
    dC_da2 = (a2 - Y)
    da2_dz2 = a2*(1.0-a2)
    delta2 = dC_da2*da2_dz2
    dC_dw2 = (delta2.T @ a1)/m
    dC_db2 = delta2.mean(axis=0)
    da1_dz1 = a1*(1.0-a1)
    delta1 = (delta2 @ w2)*da1_dz1
    dC_dw1 = (delta1.T @ X)/m
    dC_db1 = delta1.mean(axis=0)
    return delta1, delta2, dC_dw1, dC_db1, dC_dw2, dC_db2

eta = 1.0
steps3 = 220
steps_per_frame = 15

xlim = (X[:,0].min()-1, X[:,0].max()+1)
ylim = (X[:,1].min()-1, X[:,1].max()+1)

grid_x = np.linspace(xlim[0], xlim[1], 200)
grid_y = np.linspace(ylim[0], ylim[1], 200)
GX, GY = np.meshgrid(grid_x, grid_y)
G = np.stack([GX.ravel(), GY.ravel()], axis=1)

fig, (ax_dec, ax_cost) = plt.subplots(1, 2, figsize=(13, 6), gridspec_kw={"width_ratios": [3, 3]})

pos = Y.ravel() > 0.5
Z0 = sigmoid(sigmoid(G @ w1.T + b1) @ w2.T + b2).reshape(GX.shape)
img = ax_dec.imshow(Z0, extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()],
                    origin="lower", alpha=0.6, vmin=0.0, vmax=1.0, interpolation="nearest")
s1 = ax_dec.scatter(X[pos,0], X[pos,1], s=16, label="Clase 1")
s0 = ax_dec.scatter(X[~pos,0], X[~pos,1], s=16, label="Clase 0")
cs = ax_dec.contour(GX, GY, Z0, levels=[0.5], colors="k", linewidths=2)
txt = ax_dec.text(0.02, 0.98, "", transform=ax_dec.transAxes, va="top")
ax_dec.set_title("Backprop 2-2-1: evolución de la frontera p=0.5")
ax_dec.set_xlabel("x1")
ax_dec.set_ylabel("x2")
ax_dec.set_xlim(*xlim)
ax_dec.set_ylim(*ylim)
ax_dec.legend(loc="lower right")

ax_cost.set_title("Costo vs época")
ax_cost.set_xlabel("Época")
ax_cost.set_ylabel("Costo")
ax_cost.set_xlim(0, steps3)
ax_cost.set_ylim(0.05, 0.13)
line_cost, = ax_cost.plot([], [], lw=2)
dot_cost, = ax_cost.plot([], [], "o", ms=6)
cost_hist = []

def init3():
    line_cost.set_data([], [])
    dot_cost.set_data([], [])
    txt.set_text("")
    return img, s1, s0, txt, line_cost, dot_cost

def update3(t):
    global w1, b1, w2, b2
    C_show = None
    for _ in range(steps_per_frame):
        z1, a1, z2, a2 = forward_2_2_1(X, w1, b1, w2, b2)
        C = loss_mse(a2, Y)
        delta1, delta2, dC_dw1, dC_db1, dC_dw2, dC_db2 = backprop_2_2_1(X, Y, z1, a1, z2, a2, w2)
        w2 -= eta*dC_dw2
        b2 -= eta*dC_db2
        w1 -= eta*dC_dw1
        b1 -= eta*dC_db1
        C_show = C
    cost_hist.append(float(C_show))

    Zg = sigmoid(sigmoid(G @ w1.T + b1) @ w2.T + b2).reshape(GX.shape)
    ax_dec.cla()
    ax_dec.imshow(Zg, extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()],
                  origin="lower", alpha=0.6, vmin=0.0, vmax=1.0, interpolation="nearest")
    ax_dec.scatter(X[pos,0], X[pos,1], s=16, label="Clase 1")
    ax_dec.scatter(X[~pos,0], X[~pos,1], s=16, label="Clase 0")
    ax_dec.contour(GX, GY, Zg, levels=[0.5], colors="k", linewidths=2)
    ax_dec.set_title("Backprop 2-2-1: evolución de la frontera p=0.5")
    ax_dec.set_xlim(*xlim)
    ax_dec.set_ylim(*ylim)
    ax_dec.legend(loc="lower right")
    ax_dec.text(0.02, 0.98, f"Época={t+1}\nC={C_show:.4f}", transform=ax_dec.transAxes, va="top")

    xs = np.arange(1, len(cost_hist)+1)
    ys = np.array(cost_hist)
    line_cost.set_data(xs, ys)
    dot_cost.set_data([xs[-1]], [ys[-1]])

    return line_cost, dot_cost

anim3 = animation.FuncAnimation(fig, update3, init_func=init3, frames=steps3, interval=50, blit=False)
html3 = HTML(anim3.to_jshtml())
display(html3)
plt.close()

Output hidden; open in https://colab.research.google.com to view.